<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250> 
<b>Introduction to Source Detection</b> <br>
Last verified to run on <b>TBD</b> with LSST Science Pipelines release <b>TBD</b> <br>
Contact author: Alex Drlica-Wagner <br>
Credit: Originally developed in the context of the LSST Stack Club <br>
Target audience: All DP0 delegates. <br>
Container Size: medium <br>
<br>
Questions welcome at <a href="https://community.lsst.org/c/support/dp0">community.lsst.org/c/support/dp0</a> <br>
Find DP0 documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a> <br>
<br>


This notebook demonstrates how to run the source detection, measurment, and deblending algorithms with a focus on optimizing for low-surface-brightness object detection. It attempts to split out the source detection and measurement algorithms from `processCCD` and apply them to the search for low-surface-brightness galaxies. Some source detection and measurement details come from [Tune Detection.ipynb](https://github.com/RobertLuptonTheGood/notebooks/blob/master/Demos/Tune%20Detection.ipynb) and [Kron.ipynb](https://github.com/RobertLuptonTheGood/notebooks/blob/master/Demos/Kron.ipynb).
Interaction with `lsst.afw.display` was also improved by studying Michael Wood-Vasey's [DC2_Postage Stamps.ipynb](https://github.com/LSSTDESC/DC2-analysis/blob/master/tutorials/dm_butler_postage_stamps.ipynb).

### Learning Objectives:
After working through this notebook you should be able to
   1. Run the `lsst.meas.algorithm` source detection, deblending, and measurement tasks.
   2. Plot the resulting source catalogs
   3. Examine the `Footprint` of the detected sources

Other techniques that are demonstrated, but not empasized, in this notebook are
   1. Use the `butler` to access a specific `calexp`.
   2. Create an image cutout and use `lsst.afw.display` to plot it.

**Credit:** This notebook was adapted from notebooks developed by Alex Drlica-Wagner and Imran Hasan in the context of the LSST Stack Club.

### Set Up

In [ ]:
# What version of the Stack are we using?
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
%matplotlib inline
#%matplotlib ipympl # currently slow, but may be a good option in the future
import os
import warnings
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import IFrame, display, Markdown
from mpl_toolkits.axes_grid1 import make_axes_locatable

from matplotlib.patches import Rectangle
from astropy.visualization import ZScaleInterval

In [ ]:
import lsst.daf.persistence as dafPersist
import lsst.daf.base        as dafBase
import lsst.daf.butler      as dafButler

import lsst.afw.image       as afwImage
import lsst.afw.display     as afwDisplay
import lsst.afw.table       as afwTable
import lsst.geom            as afwGeom
import lsst.obs.base        as obsBase

In [ ]:
# Filter some warnings printed by v16.0 of the stack
#warnings.simplefilter("ignore", category=FutureWarning)
#warnings.simplefilter("ignore", category=UserWarning)

# Use lsst.afw.display with the matplotlib backend
afwDisplay.setDefaultBackend('matplotlib') 

zscale = ZScaleInterval()
plt.rcParams['figure.figsize'] = (8.0, 8.0)

In [ ]:
# Position of our low surface brightness "galaxy"
x_target, y_target = 1700, 2100
width,height=400,400
xmin,ymin = x_target-width//2, y_target-height//2

# Position of our cutout
#x_target, y_target = 3835, 2380
#xmin,ymin = 3550,2100


## Data access

Here we use the `butler` to access a `calexp` from the DP0.1 dataset. More information on the `butler` and `calexp` are available elsewhere, and we expect the user to have a working knowledge of these objects.

In [ ]:
from lsst.daf.butler import Butler 

# Grab a calexp
dataset_type = 'calexp'
dataId = {'filter':'i', 'visit': 512055, 'raftName': 'R20', 'detector': 75}

# DC2 gen3
repo='s3://butler-us-central1-dp01'
collection='2.2i/runs/DP0.1'

In [ ]:
butler = Butler(repo,collections=collection)
calexp = butler.get(dataset_type, **dataId)

As described elsewhere, the `calexp` object possess more than just the raw pixel data of the image. It also contains a `mask`, which stores information about various pixels in a bit mask. Since we are interested in performing our own source detection and measurement, we choose to clear the previously set `DETECTED` mask plane.

In [ ]:
# Unset the `DETECTED` bits of the mask plane
calexp.mask.removeAndClearMaskPlane('DETECTED')

In [ ]:


plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp.image)

Our next step is to generate a cutout image. This is done by creating a bounding box and passing it to the `Factory` method of our calexp (a `lsst.afw.image.Exposure` object). Unfortunately, the arguments for the `Factory` method are poorly documented, and below we explain the specific arguments that we are passing to `Factory`:
```
calexp : the ExposureF we are starting from
bbox   : the bounding box of the cutout
origin : the image pixel origin is local to the cutout array
deep   : copy the data rather than passing by reference
```

In [ ]:
# Define a small region for a cutout
bbox = afwGeom.Box2I()
bbox.include(afwGeom.Point2I(xmin, ymin))
bbox.include(afwGeom.Point2I(xmin+width, ymin+height))

# Generate the cutout image
cutout = calexp.Factory(calexp, bbox, origin=afwImage.LOCAL, deep=False)

In [ ]:
# Follow the same procedure as before to plot the cutout
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(cutout.image)
plt.gca().axis('off')

# Source Detection, Deblending, and Measurement

We now want to run the LSST source detection, deblending, and measurement tasks. While we run all three tasks, this notebook is mostly focused on the detection of faint sources.

In [ ]:
# Importing the tasks
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.pipe.tasks.calibrate         import CalibrateTask
from lsst.meas.algorithms.detection    import SourceDetectionTask
from lsst.meas.deblender               import SourceDeblendTask
from lsst.meas.base                    import SingleFrameMeasurementTask

Each task possesses an associated configuration class. The properties of these classes can be determined from the classes themselves.

In [ ]:
# Uncomment the following line to view help on the CharacterizeImageTask configuration
#help(CharacterizeImageTask.ConfigClass())

In [ ]:
# Create the Tasks
schema = afwTable.SourceTable.makeMinimalSchema()
algMetadata = dafBase.PropertyList()

config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 1
charImageTask = CharacterizeImageTask(None, config=config)

config = SourceDetectionTask.ConfigClass()
config.thresholdValue = 10       # detection threshold in units of thresholdType
config.thresholdType = "stdev"   # units for thresholdValue

sourceDetectionTask =   SourceDetectionTask(schema=schema, config=config)
sourceDeblendTask   =   SourceDeblendTask(schema=schema)

config = SingleFrameMeasurementTask.ConfigClass()
sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema, config=config,
                                                   algMetadata=algMetadata)

With the each of the tasks configured, we can now move on to running the source detection, deblending, and measurement. Like the configs, we can use `help` to explore each task and the methods used to run it.

First we create `SourceTable` for holding the output of our source analysis. The columns and characteristics of this table are defined by the `schema` that we created in our configuration step.

In [ ]:
tab = afwTable.SourceTable.make(schema)

Next we characterize our image. This calculates various global properties, such as the PSF FWHM.

In [ ]:
# Image characterization (this cell may take a few seconds)
result = charImageTask.run(calexp)

psf = calexp.getPsf()
sigma = psf.computeShape().getDeterminantRadius()
pixelScale = calexp.getWcs().getPixelScale().asArcseconds()
# The factor of 2.355 converts from std to fwhm
print('psf fwhm = {:.2f} arcsec'.format(sigma*pixelScale*2.355))

With the image characterized, we are now interested in running the source detection, deblending, and measurement tasks. Each of these tasks is called with the `run` method. The parameters of this method can be investigated using `help`.

In [ ]:
# We are specifically interested in the `SourceMeasurementTask`
#help(sourceMeasurementTask.run)

In [ ]:
# Source detection (this cell may take a few seconds)
result = sourceDetectionTask.run(tab, calexp)
type(result)

The source detection task returns an [`lsst.pipe.base.struct.Struct`](http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/classlsst_1_1pipe_1_1base_1_1struct_1_1_struct.html). A `Struct` is just a generalized container for storing multiple output components and accessessing them as attributes. The content of this `Struct` can be investigated with the `getDict` method.

In [ ]:
for k,v in result.getDict().items():
    print(k, type(v))

The members of the `Struct` can be accessed either through dictionary keys or as attributes of the `Struct`. For example:

In [ ]:
sources = result.sources

Note that if we desire we can save some of these processed objects to disk.

In [ ]:
if False:
    sources.writeFits("outputTable.fits")
    calexp.writeFits("example1-out.fits")

Next we run the `SourceDeblendTask` and `SingleFrameMeasurementTask`. A deeper investigation of these tasks is beyond the scope of this notebook.

In [ ]:
# Source deblending
sourceDeblendTask.run(calexp, sources)

# Source measurement (catch future warning about machine precision)
sourceMeasurementTask.run(measCat=sources, exposure=calexp)

To get a better look at the output sources, we need to make sure that the `SourceCatalog` is contiguous in memory. Converting to an `astropy` table provides a human-readable output format. A deeper dive into `SourceCatalog` is beyond the scope of this notebook.

In [ ]:
# The copy makes sure that the sources are sequential in memory
sources = sources.copy(True)

# Investigate the output source catalog
sources.asAstropy()

We can now overplot our detected sources on the calexp or cutout image using `afwDisplay`.

<a id='display-error'></a>

In [ ]:
#Display the cutout and sources with afw display
image = cutout.image
#image = calexp.image

plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(image)
plt.gca().axis('off')

# We use display buffering to avoid re-drawing the image after each source is plotted
with afw_display.Buffering():
    for s in sources:
        afw_display.dot('+', s.getX(), s.getY(), ctype=afwDisplay.RED)
        afw_display.dot('o', s.getX(), s.getY(), size=20, ctype='orange')   

## Footprints

To paraphrase from [Bosch et al. (2017)](https://arxiv.org/pdf/1705.06766.pdf), 

> Footprints record the exact above-threshold detection region on a CCD. These are similar to  SExtractor’s “segmentation map", in that they identify which pixels belong to which detected objects

As you might expect, this means footprints are integral to high-level CCD processing tasks&mdash;like detection, measurement, and deblending&mdash;which directly impact science results. Because footprints are so closely related to these very important processes, we will take a close look at them in this notebook.

In the quote above, an analogy was drawn between footprints and segmentation maps, as they both identify above threshold pixels. As we first introduce footprints, we will concentrate on this similarity as it gives us a place to start understanding the location and geometeric properties of footprints. 

We will use the `detectFootprints` method in `SourceDetectionTask` to find and store the detected footprints in the image

In [ ]:
# lets grab the above threshold footprints that were detected and assign them to a varriable
fpset = result.positive
fps = fpset.getFootprints()

In [ ]:
# We can get a rough view of the footprint from span
fps[0].getSpans()

You can almost see the footprint by looking at the 1's and zeros here. To extract the actual pixel values that correspond to the ones in the span, we need an additional step. At the moment, our footprints can tell you if a pixel belongs to it or not, but are not accessing pixel values on the image. To remedy this, we will turn our footprint into a `HeavyFootprint`. HeavyFootprints have all of the qualities of Footprints, but additionally 'know' about pixel level data from the image, variance, and mask planes.

In [ ]:
# first we domonstrate the footprint is not heavy
fps[0].isHeavy()

In [ ]:
# we will make all the footprints heavy at the same time by operating on the footprint set
fpset.makeHeavy(calexp.getMaskedImage())
# we have to redefine fps
hfps = fpset.getFootprints()

In [ ]:
# all of the arrays here will be flattend 1D arrays of pixels from the footprint
hfps[0].getImageArray()

Now we can use the spanset to reassemble the image array into the footprint. Above we saw that the image array is a 1D numpy array-but the footprint itself is 2 dimensional. Fortunately, the span set has an `unflatten` method that we will use, which can rearrange the image array into the proper 2 dimensional shape

In [ ]:
plt.imshow(fps[0].getSpans().unflatten(hfps[0].getImageArray()),
           cmap='bone', origin='lower')

In [ ]:
hfps[0].getMaskArray()

To understand these values, lets look at the mask plane's dictionary

In [ ]:
calexp.getMask().getMaskPlaneDict()

The values are the exponent of the bitmask. So pixels only marked detected will be 2^5 = 32. Pixels that are both on the edge and detected will be 2^5 + 2^4 = 48. Now we will visualize this in a similar manner to the imshow exercise we did before, only now we are *only* using data for the footprint because we are using the span.

In [ ]:
plt.figure(figsize=(8,8))
ax = plt.gca()
# create an axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.
divider = make_axes_locatable(ax)
im = plt.imshow(fps[0].getSpans().unflatten(hfps[0].getMaskArray()),
                origin='lower')

cax = divider.append_axes("right", size="5%", pad=0.05)

plt.colorbar(im, cax=cax, ticks=[0, 32, 32+16])

More information on footprints can be found on the Stack Club notebook by Imran Hasan [here](https://github.com/LSSTScienceCollaborations/StackClub/blob/master/SourceDetection/Footprints.ipynb).

# Summary

In this tutorial you should have learned how to run several tasks to perform source detection and measurement. We have then investigated the footprints associated with detected objects.